This Notebook helps transforming the V1 Beschlüsse to new V2 Resolutions DB Schema


In [27]:
import pymongo
from dotenv import load_dotenv
import os
from dataclasses import dataclass
load_dotenv()
import secrets
import datetime
import pprint
import json

MONGO_URI = os.getenv('MONGO_URI')

pp = pprint.PrettyPrinter(indent=4)


@dataclass
class Beschluss:
    id: str
    titel: str
    tag: str
    antragsteller: str
    fließtext: str
    jahr: int

# @dataclass
# class Resolution:
#     rid: str
#     rcode: str
#     created: int
#     user: str
#     parent: str
#     body: {
#         tag: str
#         antragsteller: str
#         fließtext: str
#         jahr: int
#         resolution: str
#     }





In [28]:
# Connect to MongoDB
client = pymongo.MongoClient(MONGO_URI)
beschlussCol = client['beschlusswiki']['beschlüsse']
resolutionCol = client['beschlusswiki']['resolutions']

RID_LENGTH = 2

used_ids = []



def createResolution(beschluss):
    # Create rid: unique random 4 digit hex, used ids are stored in used_ids
    rid = secrets.token_hex(RID_LENGTH)
    while rid in used_ids:
        rid = secrets.token_hex(RID_LENGTH)

    rcode = str(beschluss.jahr) + "-" + beschluss.tag
    # Created unix timestamp in milliseconds
    created = int(datetime.datetime.now().timestamp() * 1000)  
    applicants = []
    if beschluss.antragsteller:
        antragssteller = beschluss.antragsteller
        applicants.append(antragssteller)
    
    resolution = {
        "rid": rid,
        "rcode": rcode,
        "created": created,
        "user": "admin",
        "parent": None,
        "body": {
            "title": beschluss.titel,
            "tag": beschluss.tag,
            "applicants": applicants,
            "year": beschluss.jahr,
            "category": {
                "id": "ffff",
                "name": "ffff",
            },
            "text": beschluss.fließtext,
        }
    }
    return resolution


# Get all beschlüssse
beschlüsse = beschlussCol.find()

resolutions = []

# Loop through all beschlüsse
for beschluss in beschlüsse:
    try:
        bv1 = Beschluss(
            id= beschluss['_id'].__str__(),
            tag=beschluss['tag'],
            titel=beschluss['titel'],
            antragsteller=getattr(beschluss, 'antragsteller', None),
            fließtext=beschluss['fließtext'],
            jahr=beschluss['jahr'],
        )
    except KeyError as e:
        print("Beschluss {} enthält Fehler".format(beschluss['_id'].__str__()))
        print(e)
        pass
    res = createResolution(bv1)
    j = json.dumps(res)
    resolutions.append(res)

print("{} Beschlüsse gefunden und konvertiert".format(len(resolutions)))



with open('resolutions.json', 'w', encoding="utf-8") as f:
    json.dump(resolutions, f, ensure_ascii=False, indent=4)

print("Beschlüsse in resolutions.json gespeichert")

# Push resolutions to MongoDB

res = resolutionCol.insert_many(resolutions)
print("{} Resolutions in MongoDB gespeichert".format(len(res.inserted_ids)))



    

Beschluss 65199beb05fb232cb02ac446 enthält Fehler
'tag'
Beschluss 65199ce705fb232cb02ac44a enthält Fehler
'tag'
59 Beschlüsse gefunden und konvertiert
Beschlüsse in resolutions.json gespeichert
